# Initial Data Exploration

This notebook contains the initial exploration of the TailWagg pet retail dataset, including:
- Database connection setup
- Basic data loading and inspection
- Initial product analysis
- Data quality checks


## Setup


In [1]:
# Add project root to Python path
import sys
import os

# Get the project root directory (parent of notebooks directory)
current_dir = os.getcwd()
if current_dir.endswith('notebooks'):
    project_root = os.path.dirname(current_dir)
else:
    project_root = current_dir

# Add project root to Python path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")
print(f"Python path includes: {project_root in sys.path}")

# Import TailWagg utilities
from src.utils.database import get_database_engine, test_connection
from src.dataset import load_daily_metrics
from src.utils.validation import validate_environment
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Project root: /Users/paulrodriguez/Documents/Documents - Paul’s MacBook Pro/Data Analyst School/_DataCamp/github/tailwagg
Python path includes: True


### Configure SQLAlchemy Connection


In [3]:
# Validate environment and get database engine
validate_environment()
engine = get_database_engine()

# Test connection
if test_connection(engine):
    print("Connected to PostgreSQL successfully!")
else:
    print("Failed to connect to PostgreSQL")


Connected to PostgreSQL successfully!


## Initial Data Exploration


In [4]:
# Load daily metrics using TailWagg utility
df = load_daily_metrics()

# Convert order_date to datetime for proper date handling
df['order_date'] = pd.to_datetime(df['order_date'])

print("Daily metrics data loaded:")
print(f"Shape: {df.shape}")
print(f"Date range: {df['order_date'].min()} to {df['order_date'].max()}")
print(f"Unique products: {df['product_id'].nunique()}")
print(f"Unique categories: {df['category_name'].nunique()}")
print("\nData summary:")
print(df.describe().round(2))


Daily metrics data loaded:
Shape: (216425, 9)
Date range: 2022-10-17 00:00:00 to 2025-10-16 00:00:00
Unique products: 457
Unique categories: 5

Data summary:
                          order_date  total_units_sold  gross_revenue  \
count                         216425         216425.00      216425.00   
mean   2024-04-05 14:10:24.005544448              1.78          61.50   
min              2022-10-17 00:00:00              1.00           1.67   
25%              2023-07-22 00:00:00              1.00          22.95   
50%              2024-04-20 00:00:00              2.00          44.84   
75%              2024-12-05 00:00:00              2.00          80.14   
max              2025-10-16 00:00:00             11.00         690.02   
std                              NaN              0.96          54.92   

       total_discount       cogs  gross_profit  
count       216425.00  216425.00     216425.00  
mean             7.77      48.50         13.00  
min              0.00       1.83     

In [5]:
# Get sample products from the loaded data
print("Sample products from daily metrics:")
sample_products = df[['product_id', 'category_name']].drop_duplicates().head(10)
print(sample_products)


Sample products from daily metrics:
     product_id category_name
0      prod_001        Treats
501    prod_002        Treats
999    prod_003      Wellness
1467   prod_006   Accessories
1954   prod_007      Wellness
2410   prod_008        Treats
2878   prod_009          Toys
3346   prod_011   Accessories
3807   prod_012   Accessories
4250   prod_015          Toys


### Basic Data Quality Checks


In [6]:
# Check for missing values
print("Missing values in daily metrics data:")
print(df.isnull().sum())

print("\nData types:")
print(df.dtypes)

print("\nUnique values in key columns:")
print(f"Unique products: {df['product_id'].nunique()}")
print(f"Date range: {df['order_date'].min()} to {df['order_date'].max()}")
print(f"Unique categories: {df['category_name'].nunique()}")
print(f"Unique promotions: {df['promo_id'].nunique()}")


Missing values in daily metrics data:
product_id              0
category_name           0
promo_id            85437
order_date              0
total_units_sold        0
gross_revenue           0
total_discount          0
cogs                    0
gross_profit            0
dtype: int64

Data types:
product_id                  object
category_name               object
promo_id                    object
order_date          datetime64[ns]
total_units_sold             int64
gross_revenue              float64
total_discount             float64
cogs                       float64
gross_profit               float64
dtype: object

Unique values in key columns:
Unique products: 457
Date range: 2022-10-17 00:00:00 to 2025-10-16 00:00:00
Unique categories: 5
Unique promotions: 27


## Export Database Tables to CSV

Export all dimension and fact tables to the `data/raw/` directory for backup and version control.


In [7]:
# Export all database tables to CSV files
# Ensure the data/raw directory exists
os.makedirs(os.path.join(project_root, 'data', 'raw'), exist_ok=True)

# Define all tables to export
dimension_tables = [
    'dim_category',
    'dim_brand', 
    'dim_channel',
    'dim_location',
    'dim_customer',
    'dim_promo',
    'dim_product'
]

fact_tables = [
    'fact_sales',
    'fact_ad_spend',
    'fact_email',
    'fact_inventory_snapshots',
    'fact_returns'
]

all_tables = dimension_tables + fact_tables

# Export each table to CSV
print("Exporting database tables to CSV...")
print("="*60)

for table_name in all_tables:
    try:
        # Read table from database
        query = f"SELECT * FROM {table_name}"
        df_table = pd.read_sql(query, engine)
        
        # Define output path
        output_path = os.path.join(project_root, 'data', 'raw', f'{table_name}.csv')
        
        # Export to CSV
        df_table.to_csv(output_path, index=False)
        
        print(f"✅ {table_name}: {len(df_table):,} rows exported to {table_name}.csv")
        
    except Exception as e:
        print(f"❌ {table_name}: Export failed - {e}")

print("="*60)
print("Export complete!")


Exporting database tables to CSV...
✅ dim_category: 5 rows exported to dim_category.csv
✅ dim_brand: 8 rows exported to dim_brand.csv
✅ dim_channel: 7 rows exported to dim_channel.csv
✅ dim_location: 5 rows exported to dim_location.csv
✅ dim_customer: 6,000 rows exported to dim_customer.csv
✅ dim_promo: 39 rows exported to dim_promo.csv
✅ dim_product: 600 rows exported to dim_product.csv
✅ fact_sales: 246,369 rows exported to fact_sales.csv
✅ fact_ad_spend: 2,192 rows exported to fact_ad_spend.csv
✅ fact_email: 3,801 rows exported to fact_email.csv
✅ fact_inventory_snapshots: 78,500 rows exported to fact_inventory_snapshots.csv
✅ fact_returns: 17,871 rows exported to fact_returns.csv
Export complete!
